In [ ]:
from agential.cog.prompts.critic import (
    ################### HUMANEVAL ###################

    # Main prompt prompt/fewshot examples.
    CRITIC_POT_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_POT,

    CRITIC_POT_INSTRUCTION_WITH_TESTS_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_POT_WITH_TESTS,

    # Critic prompt/fewshot examples.
    CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC,
    
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,

    # Generate unit tests prompt/fewshot examples.
    CRITIC_POT_INSTRUCTION_TEST_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_POT_TEST,

    ################### MBPP ###################

    # Main prompt/fewshot examples.
    CRITIC_POT_INSTRUCTION_WITH_TESTS_MBPP,
    MBPP_FEWSHOT_EXAMPLES_POT_WITH_TESTS,

    # Critic prompt/fewshot examples.
    CRITIC_CRITIQUE_INSTRUCTION_MBPP,
    MBPP_FEWSHOT_EXAMPLES_CRITIC,  # TODO

    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_MBPP,  # TODO
    MBPP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL  # TODO
)
from agential.cog.agent.critic import CriticAgent
from langchain_community.chat_models.openai import ChatOpenAI

import warnings
warnings.filterwarnings("ignore")

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()

In [ ]:
from agential.cog.functional.critic import safe_execute

a = """import heapq as hq

def heap_queue_largest(nums, n):
    return sorted(nums)[-n:]

assert heap_queue_largest([25, 35, 22, 85, 14, 65, 75, 22, 58], 3) == [85, 75, 65], "Test failed for top 3"
assert heap_queue_largest([25, 35, 22, 85, 14, 65, 75, 22, 58], 2) == [85, 75], "Test failed for top 2"
assert heap_queue_largest([25, 35, 22, 85, 14, 65, 75, 22, 58], 5) == [85, 75, 65, 58, 35], "Test failed for top 5"
"""

safe_execute(a)

In [ ]:
agent = CriticAgent(
    llm=llm,
    mode="code"
)

In [ ]:
question = """from typing import List 

def has_close_elements(numbers: List[float], threshold: float) -> bool: 
    \"\"\"Check if in given list of numbers, are any two numbers closer to each other than given threshold. 
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5) False 
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) True
    \"\"\"
"""

tests = """assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True, "THIS TEST IS FAILING"
assert has_close_elements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
assert has_close_elements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False
assert has_close_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True
assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True
assert has_close_elements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False
"""

In [3]:
import builtins
import sys

def exe():
    exec(
        """from collections import Counter

def function():
    a = Counter("bbbb")
    print('Hello Worlds!')

function()
""", {"__builtin__": builtins, "sys": sys}
    )

In [4]:
exe()

Hello Worlds!


In [ ]:
out = agent.generate(
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT_WITH_TESTS,
    prompt=CRITIC_POT_INSTRUCTION_WITH_TESTS_HUMANEVAL,
    additional_keys={"tests": tests},
    critique_examples=HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL,
    use_interpreter_tool=True,
    tests=tests,
    test_examples="",
    test_prompt=""
)